In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('AirPort').getOrCreate()


In [9]:
book_df = spark.read.options(sep=';', header=True, inferSchema=True).csv("./BooksRating-CSV/Books.csv").select('ISBN', 'BookTitle')
user_df = spark.read.options(sep=';', header=True, inferSchema=True).csv("./BooksRating-CSV/Users.csv").select('UserID', 'USERNAME')
rate_df = spark.read.options(sep=';', header=True, inferSchema=True).csv("./BooksRating-CSV/Book-Ratings.csv")
rate_df.show(5)

+------+----------+----+
|userid|      isbn|rate|
+------+----------+----+
|276725|034545104X|   0|
|276726|0155061224|   5|
|276727|0446520802|   0|
|276729|052165615X|   3|
|276729|0521795028|   6|
+------+----------+----+
only showing top 5 rows



In [10]:
print(f"book Count: {book_df.count()}, User Count: {user_df.count()}, Rate Count: {rate_df.count()}")

book Count: 271379, User Count: 278698, Rate Count: 1149780


In [11]:
rate_avg_df = rate_df.groupBy('isbn').avg('rate')
rate_avg_df.show(10)

+----------+------------------+
|      isbn|         avg(rate)|
+----------+------------------+
|2080674722|3.6666666666666665|
|3499134004|               0.0|
|3548603203|3.4166666666666665|
|880781112X| 4.333333333333333|
|0738205737|1.6666666666666667|
|0749317256|               0.0|
|0515131520|            1.6875|
|8471662531|               7.0|
|0441005470|               0.0|
|0739417096| 5.285714285714286|
+----------+------------------+
only showing top 10 rows



In [15]:
rate_df = rate_df.join(rate_avg_df, on="isbn", how="left_outer")
rate_df.show(10)

+----------+------+----+------------------+
|      isbn|userid|rate|         avg(rate)|
+----------+------+----+------------------+
|0373241801|106936|   0|               0.0|
|0380718421|261275|   7|1.7619047619047619|
|0425190048|261274|  10|               5.0|
|0767915054|211919|   0| 2.480769230769231|
|0787602035|211919|   6|               6.0|
|2080674722|276733|   0|3.6666666666666665|
| 342310538|276745|  10|               5.0|
|3442545668|160406|   0|               0.0|
|034545104X|276725|   0| 2.933333333333333|
|0373240414|106936|   0|               0.0|
+----------+------+----+------------------+
only showing top 10 rows



In [17]:
df = rate_df.join(user_df, on='userid', how="left_outer")
df.show(10)

+------+----------+----+------------------+-----------+
|userid|      isbn|rate|         avg(rate)|   USERNAME|
+------+----------+----+------------------+-----------+
|106936|0373241801|   0|               0.0|LSDkMeABeWB|
|261275|0380718421|   7|1.7619047619047619|FJsT5EMvG4I|
|261274|0425190048|  10|               5.0|5hUaGV0wSbQ|
|211919|0767915054|   0| 2.480769230769231|zBAXN2eueeL|
|211919|0787602035|   6|               6.0|zBAXN2eueeL|
|276733|2080674722|   0|3.6666666666666665|imIbzo4rK7T|
|276745| 342310538|  10|               5.0|jLRV9LYfEUZ|
|160406|3442545668|   0|               0.0|CVEPW5bg8fP|
|276725|034545104X|   0| 2.933333333333333|qP0KgArgnkK|
|106936|0373240414|   0|               0.0|LSDkMeABeWB|
+------+----------+----+------------------+-----------+
only showing top 10 rows



In [18]:
df = df.join(book_df, on='isbn', how="left_outer")
df.show(10)

+----------+------+----+------------------+-----------+--------------------+
|      isbn|userid|rate|         avg(rate)|   USERNAME|           BookTitle|
+----------+------+----+------------------+-----------+--------------------+
|0373241801|106936|   0|               0.0|LSDkMeABeWB|For The Love Of S...|
|0380718421|261275|   7|1.7619047619047619|FJsT5EMvG4I|Name Withheld: A ...|
|0425190048|261274|  10|               5.0|5hUaGV0wSbQ|A Gathering Place...|
|0767915054|211919|   0| 2.480769230769231|zBAXN2eueeL|A Girl Named Zipp...|
|0787602035|211919|   6|               6.0|zBAXN2eueeL|Wild Planet! 1,00...|
|2080674722|276733|   0|3.6666666666666665|imIbzo4rK7T|Les Particules El...|
| 342310538|276745|  10|               5.0|jLRV9LYfEUZ|                NULL|
|3442545668|160406|   0|               0.0|CVEPW5bg8fP|                NULL|
|034545104X|276725|   0| 2.933333333333333|qP0KgArgnkK|Flesh Tones: A Novel|
|0373240414|106936|   0|               0.0|LSDkMeABeWB|Bachelor And The ...|

In [20]:
df.select('USERNAME', 'BookTitle', 'rate', 'avg(rate)').show(20)

+-----------+--------------------+----+------------------+
|   USERNAME|           BookTitle|rate|         avg(rate)|
+-----------+--------------------+----+------------------+
|Eg2O6NDkj3d|Little Altars Eve...|   9|3.6363636363636362|
|LSDkMeABeWB|For The Love Of S...|   0|               0.0|
|LSDkMeABeWB|                NULL|   0|               0.0|
|FJsT5EMvG4I|Name Withheld: A ...|   7|1.7619047619047619|
|5hUaGV0wSbQ|A Gathering Place...|  10|               5.0|
|3FDoZqpSivc|  Hello from Heaven!|   7|             2.625|
|zBAXN2eueeL|A Girl Named Zipp...|   0| 2.480769230769231|
|7uxi3eD6COS|         At the Edge|   0|3.2857142857142856|
|zBAXN2eueeL|Wild Planet! 1,00...|   6|               6.0|
|zBAXN2eueeL|Yesterday's Dream...|   0|               0.0|
|zBAXN2eueeL|Online Markets fo...|   0|               0.0|
|zBAXN2eueeL|In This Dark Hous...|   0|               0.0|
|zBAXN2eueeL|The Golden Mean: ...|  10| 5.944444444444445|
|riY2oKElGRk|Little Clown Car'...|   5|               5.

In [21]:
print(df.count())

1149780
